In [2]:
import pandas as pd
import numpy as np

# Generate 1,000 samples from a normal distribution
heights = np.random.normal(loc=170, scale=10, size=1000)
weights = np.random.normal(loc=70, scale=15, size=1000)

# Create a correlation between height and weight
weights += (heights - 170) * 0.5
df = pd.DataFrame({'Height': heights, 'Weight': weights})

# Add categorical data
genders = np.random.choice(['Male', 'Female'], size=1000)
df['Gender'] = genders
df.head()

,Height,Weight,Gender
0,180.642343,71.796695,Female
1,159.671988,69.360861,Male
2,173.070174,74.058238,Male
3,173.718081,55.682485,Female
4,166.712283,40.192593,Male


In [3]:
import numpy as np
import pandas as pd

def generate_customer_data(num_records):
    data = []
    for _ in range(num_records):
        age = np.random.randint(18, 80)
       
        # Rule: Income is loosely based on age
        base_income = 20000 + (age - 18) * 1000
        income = np.random.normal(base_income, base_income * 0.2)
       
        # Rule: Credit score is influenced by age and income
        credit_score = min(850, max(300, int(600 + (age/80)*100 + (income/100000)*100 + np.random.normal(0, 50))))
       
        # Rule: Loan amount is based on income and credit score
        max_loan = income * (credit_score / 600)
        loan_amount = np.random.uniform(0, max_loan)
       
        data.append([age, income, credit_score, loan_amount])
   
    return pd.DataFrame(data, columns=['Age', 'Income', 'CreditScore', 'LoanAmount'])

df = generate_customer_data(1000)
df.head()

,Age,Income,CreditScore,LoanAmount
0,34,47571.718935,673,7598.086660
1,25,23328.309474,680,23748.329317
2,45,47630.987937,661,19580.840597
3,32,33707.101907,676,1523.866827
4,76,64284.428077,772,64521.395181


In [7]:
import simpy
import random
import pandas as pd

class Bank(object):
    def __init__(self, env, num_tellers):
        self.env = env
        self.teller = simpy.Resource(env, num_tellers)
       
    def service(self, customer):
        service_time = random.expovariate(1/10)  # Avg service time of 10 minutes
        yield self.env.timeout(service_time)

def customer(env, name, bank, data):
    arrival_time = env.now
    print(f'{name} arrives at the bank at {arrival_time:.2f}')
   
    with bank.teller.request() as request:
        yield request
        wait_time = env.now - arrival_time
        print(f'{name} waits for {wait_time:.2f}')
       
        yield env.process(bank.service(name))
       
        service_time = env.now - arrival_time
        print(f'{name} leaves the bank at {env.now:.2f}')
   
    # Append the customer data to the list
    data.append((name, arrival_time, wait_time, service_time))

def run_simulation(env, num_customers, bank, data):
    for i in range(num_customers):
        env.process(customer(env, f'Customer {i}', bank, data))
        yield env.timeout(random.expovariate(1/5))  # New customer every 5 minutes on average

# Set up and run the simulation
env = simpy.Environment()
bank = Bank(env, num_tellers=3)
num_customers = 100
data = []  # This will hold the customer data
env.process(run_simulation(env, num_customers, bank, data))
env.run()

# Create a DataFrame from the collected data
df = pd.DataFrame(data, columns=['Customer', 'ArrivalTime', 'WaitTime', 'ServiceTime'])
df.head()

Customer 0 arrives at the bank at 0.00
Customer 0 waits for 0.00
Customer 1 arrives at the bank at 0.78
Customer 1 waits for 0.00
Customer 1 leaves the bank at 2.21
Customer 2 arrives at the bank at 8.43
Customer 2 waits for 0.00
Customer 2 leaves the bank at 11.39
Customer 0 leaves the bank at 11.96
Customer 3 arrives at the bank at 13.22
Customer 3 waits for 0.00
Customer 3 leaves the bank at 14.48
Customer 4 arrives at the bank at 19.46
Customer 4 waits for 0.00
Customer 5 arrives at the bank at 20.04
Customer 5 waits for 0.00
Customer 6 arrives at the bank at 20.88
Customer 6 waits for 0.00
Customer 6 leaves the bank at 23.59
Customer 4 leaves the bank at 26.04
Customer 5 leaves the bank at 27.51
Customer 7 arrives at the bank at 30.80
Customer 7 waits for 0.00
Customer 7 leaves the bank at 34.75
Customer 8 arrives at the bank at 35.05
Customer 8 waits for 0.00
Customer 9 arrives at the bank at 40.35
Customer 9 waits for 0.00
Customer 9 leaves the bank at 46.61
Customer 8 leaves th

,Customer,ArrivalTime,WaitTime,ServiceTime
0,Customer 1,0.778601,0.0,1.435578
1,Customer 2,8.427726,0.0,2.960750
2,Customer 0,0.000000,0.0,11.958757
3,Customer 3,13.218346,0.0,1.263081
4,Customer 6,20.882672,0.0,2.708268
